#Challenge

In [ ]:
# If additional packages are needed but are not installed by default, uncomment the last two lines of this cell
# and replace <package list> with a list of additional packages.
# This will ensure the notebook has all the dependencies and works everywhere

#import sys
#!{sys.executable} -m pip install <package list>

In [ ]:
import sys
!{sys.executable} -m pip install wget
!{sys.executable} -m pip install shutil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil


In [ ]:
# Libraries
import pandas as pd
import os
from shutil import unpack_archive
pd.set_option("display.max_columns", 101)

## Dataset

In [ ]:
# Run this cell block to download and extract image data
!wget 'https://hr-projects-assets-prod.s3.amazonaws.com/1iaanii247i/8c7bc0c59ee6bcdb3646a1997606a9d0/test.zip'
!wget 'https://hr-projects-assets-prod.s3.amazonaws.com/1iaanii247i/ffe8804da24b94ee410a8730ba297dfc/train_0.zip'
!wget 'https://hr-projects-assets-prod.s3.amazonaws.com/1iaanii247i/5460fcc87e2d7f7e19c17f719e0df00a/train_1.zip'

print('Extracting Train Dataset : Label 1')
unpack_archive('train_1.zip', '')

print('Extracting Train Dataset : Label 0')
unpack_archive('train_0.zip', '')

print('Extracting Test Dataset')
unpack_archive('test.zip', '')

print('Dataset Extracted')


os.remove('train_1.zip')
os.remove('train_0.zip')
os.remove('test.zip')
print('Extra Files Removed')

--2022-06-21 14:50:31--  https://hr-projects-assets-prod.s3.amazonaws.com/1iaanii247i/8c7bc0c59ee6bcdb3646a1997606a9d0/test.zip
Resolving hr-projects-assets-prod.s3.amazonaws.com (hr-projects-assets-prod.s3.amazonaws.com)... 54.231.128.17
Connecting to hr-projects-assets-prod.s3.amazonaws.com (hr-projects-assets-prod.s3.amazonaws.com)|54.231.128.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58948996 (56M) [application/zip]
Saving to: ‘test.zip’

test.zip            100%[===================>]  56.22M   101MB/s    in 0.6s    

2022-06-21 14:50:31 (101 MB/s) - ‘test.zip’ saved [58948996/58948996]

--2022-06-21 14:50:31--  https://hr-projects-assets-prod.s3.amazonaws.com/1iaanii247i/ffe8804da24b94ee410a8730ba297dfc/train_0.zip
Resolving hr-projects-assets-prod.s3.amazonaws.com (hr-projects-assets-prod.s3.amazonaws.com)... 52.216.88.227
Connecting to hr-projects-assets-prod.s3.amazonaws.com (hr-projects-assets-prod.s3.amazonaws.com)|52.216.88.227|:443... conne

## Data Description

Column | Description
:---|:---
`file` | Image name
`label` | Target Label (1/0)

In [ ]:
print(len(os.listdir('test')))
print(len(os.listdir('/content/train/0')))
print(len(os.listdir('/content/train/1')))

1401
2026
1989


In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPooling2D,Dropout

In [ ]:
model=Sequential()
model.add(Conv2D(32,(5,5),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10,activation="relu"))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,validation_split=0.20)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#Test directory
%mkdir Test

mkdir: cannot create directory ‘Test’: File exists


In [ ]:
#moving the test images to Test directory
!mv /content/test/  /content/Test

mv: cannot move '/content/test/' to '/content/Test/test': Directory not empty


In [ ]:
train_generator = train_datagen.flow_from_directory(
        '/content/train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary',
        subset='training'
        )

validation_generator = train_datagen.flow_from_directory(
        '/content/train',
         target_size=(64, 64),
         batch_size=32,
        class_mode='binary',
         subset='validation'
        )


test_generator = test_datagen.flow_from_directory(
    '/content/Test',
        target_size=(64, 64),
        batch_size=2,
        shuffle=False,
        class_mode='binary'
        )

Found 3213 images belonging to 2 classes.
Found 802 images belonging to 2 classes.
Found 1400 images belonging to 1 classes.


In [ ]:
type(test_generator)

keras.preprocessing.image.DirectoryIterator

In [ ]:
STEPS_PER_EPOCH = train_generator.n//train_generator.batch_size
VALIDATION_STEPS=validation_generator.n//train_generator.batch_size

In [ ]:
model.fit(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=VALIDATION_STEPS)

Epoch 1/10
100/100 [==============================] - 22s 207ms/step - loss: 7.6177 - accuracy: 0.0050 - val_loss: 7.6812 - val_accuracy: 0.0000e+00
Epoch 2/10
100/100 [==============================] - 22s 222ms/step - loss: 7.6596 - accuracy: 3.1437e-04 - val_loss: 7.6709 - val_accuracy: 0.0000e+00
Epoch 3/10
100/100 [==============================] - 21s 212ms/step - loss: 7.6712 - accuracy: 0.0025 - val_loss: 7.6709 - val_accuracy: 0.0000e+00
Epoch 4/10
100/100 [==============================] - 20s 202ms/step - loss: 7.6622 - accuracy: 0.0063 - val_loss: 7.6737 - val_accuracy: 0.0000e+00
Epoch 5/10
100/100 [==============================] - 20s 202ms/step - loss: 7.6790 - accuracy: 0.0038 - val_loss: 7.6812 - val_accuracy: 0.0000e+00
Epoch 6/10
100/100 [==============================] - 21s 213ms/step - loss: 7.6748 - accuracy: 9.4310e-04 - val_loss: 7.6812 - val_accuracy: 0.0000e+00
Epoch 7/10
100/100 [==============================] - 21s 213ms/step - loss: 7.6855 - accuracy: 0.

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation, BatchNormalization

In [ ]:
mod = Sequential()
mod.add(Conv2D(32,kernel_size=(5,5),activation='relu',input_shape=(64,64,3)))
mod.add(BatchNormalization())
mod.add(MaxPooling2D(pool_size=(2,2),padding='same'))
mod.add(Dropout(0.3))
mod.add(Conv2D(64,kernel_size=(5,5),activation='relu',padding='same'))
mod.add(BatchNormalization())
mod.add(MaxPooling2D(pool_size=(2,2),padding='same'))
mod.add(Dropout(0.3))
mod.add(Conv2D(128,kernel_size=(5,5),activation='relu',padding='same'))
mod.add(BatchNormalization())
mod.add(MaxPooling2D(pool_size=(2,2),padding='same'))
mod.add(Dropout(0.5))
mod.add(Flatten())
mod.add(Dense(units = 512,activation='relu'))
mod.add(BatchNormalization())
mod.add(Dropout(0.5))
mod.add(Dense(units = 128,activation='relu'))
mod.add(Dropout(0.25))
mod.add(Dense(10,activation='softmax'))

In [ ]:
mod.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=mod.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

700/700 [==============================] - 8s 10ms/step


In [ ]:
pred

array([[0.10065376, 0.11843435, 0.10939975, ..., 0.08541183, 0.10146184,
        0.0922607 ],
       [0.0984862 , 0.11020761, 0.11028659, ..., 0.08385603, 0.10714349,
        0.09424312],
       [0.09804894, 0.11859586, 0.10958697, ..., 0.08497509, 0.10363305,
        0.09140588],
       ...,
       [0.09202611, 0.11882417, 0.11052363, ..., 0.08755437, 0.10637037,
        0.0936203 ],
       [0.09707569, 0.11060058, 0.10815649, ..., 0.08603349, 0.10554751,
        0.09510016],
       [0.09704078, 0.11729858, 0.10972477, ..., 0.08724363, 0.10252952,
        0.0921993 ]], dtype=float32)

In [ ]:
len(pred)

1400

In [ ]:
labels_test_predictions = pd.DataFrame(pred)
labels_test_predictions

,0,1,2,3,4,5,6,7,8,9
0,0.100654,0.118434,0.109400,0.086659,0.106615,0.106007,0.093096,0.085412,0.101462,0.092261
1,0.098486,0.110208,0.110287,0.090312,0.111062,0.103080,0.091323,0.083856,0.107143,0.094243
2,0.098049,0.118596,0.109587,0.089096,0.106426,0.104795,0.093437,0.084975,0.103633,0.091406
3,0.097487,0.111763,0.106916,0.090945,0.106625,0.102353,0.095167,0.090787,0.103961,0.093995
4,0.098679,0.111662,0.110108,0.088507,0.108940,0.105280,0.094047,0.087624,0.102285,0.092869
...,...,...,...,...,...,...,...,...,...,...
1395,0.097888,0.117334,0.109235,0.090159,0.105658,0.103159,0.094522,0.086987,0.102406,0.092652
1396,0.100891,0.115073,0.109286,0.090105,0.105876,0.105182,0.091993,0.087342,0.101410,0.092842
1397,0.092026,0.118824,0.110524,0.090482,0.103040,0.104700,0.092859,0.087554,0.106370,0.093620
1398,0.097076,0.110601,0.108156,0.091402,0.112103,0.099504,0.094478,0.086033,0.105548,0.095100


In [ ]:
labels_test_predictions.to_csv('Distracted_Driver_MultiAction_Classification.csv',index=False)

In [ ]:
from google.colab import files
files.download("Distracted_Driver_MultiAction_Classification.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# The train label dataset is already loaded below
data = pd.read_csv("/content/train")
len(data)

In [ ]:
data.head()

## Machine Learning

Build a neural network that can classify the driver’s behaviour.
- **The model's performance will be evaluated on the basis of accuracy.**

> #### Task:
- **Submit the predictions on the test dataset using your optimized model** <br/>
    For each image in the test folder, predict the value of the label (0/1). You should submit a CSV file with a header row and one row per test entry. 

The file (`submissions.csv`) should have exactly 2 columns:
   - **file**
   - **label**
   

In [ ]:
#Submission
submission_df.to_csv('submissions.csv',index=False)